In [33]:
# ref  https://docs.llamaindex.ai/en/stable/examples/retrievers/recurisve_retriever_nodes_braintrust/

In [14]:
# %pip install llama-index-embeddings-together
# %pip install llama-index-llms-openai
# %pip install llama-index-embeddings-openai
# %pip install llama-index-readers-file
# %pip install llama-index-llms-openai
# %pip install llama-index-readers-file
# %pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision
# %pip install llama-index-embeddings-huggingface



In [17]:
# !pip install llama-index-core
# !pip install llama-index-llms-openai
# !pip install llama-index-postprocessor-rankllm-rerank

In [16]:
#!pip install --no-binary :all: nmslib
#!CFLAGS="-mavx -DWARN(a)=(a)" pip install nmslib
#CFLAGS="-mavx -DWARN(a)=(a)" pip install nmslib
#CFLAGS="-mavx -DWARN(a)=(a)" pip install --use-pep517 nmslib
# CFLAGS="-mavx -DWARN(a)=(a)" pip install 'rectools[nmslib]' 

# THIS IS WHAT WORKED: https://github.com/nmslib/nmslib/issues/538
# !pip install --upgrade pybind11        # 2.10.1 or higher  (latest as of today: 2.11.1)
# !pip install --verbose  'nmslib @ git+https://github.com/nmslib/nmslib.git#egg=nmslib&subdirectory=python_bindings'

In [11]:
# !pip install rank-llm


In [12]:
# %pip install -q cohere llama-index-postprocessor-cohere-rerank
# %pip install llama-index-postprocessor-rankllm-rerank


In [1]:
import sys
import os
import pandas as pd

# Add the parent directory containing both 'EDA' and 'RAG' to sys.path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Adjust this based on your project structure
sys.path.append(parent_dir)

import pandas as pd
from src_rag.document_loader import DocumentLoader
from src_rag.index_manager import IndexManager
from RAG.src_rag.text_data_handler import TextDataHandler


%load_ext autoreload
%autoreload 2




In [2]:
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document

from llama_index.core.embeddings import resolve_embed_model

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode



In [3]:
# Load the Excel file
file_path = "../data/processed_data/df_cluster.xlsx"
df = pd.read_excel(file_path)

# Check if the 'text' column exists
if 'text' not in df.columns:
    raise ValueError("The 'text' column is not present in the provided Excel file.")


# Create Document objects from the 'text' column
docs = [Document(text=row['text']) for _, row in df.iterrows()]

# get API key and create embeddings
from llama_index.embeddings.openai import OpenAIEmbedding

node_parser = SentenceSplitter(chunk_size=1024)

base_nodes = node_parser.get_nodes_from_documents(docs)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

embed_model = OpenAIEmbedding(model="text-embedding-3-large")


#embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")

base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)

base_retriever = base_index.as_retriever(similarity_top_k=5)


retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for keykundra"
)

for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-23<br>**Similarity:** 0.3156899788929284<br>**Text:** How do healthcare providers educate patients about the benefits and risks of Keytruda? Healthcare providers offer educational materials, detailed consultations, and regular follow-up to discuss treatment progress and side effects.<br>

**Node ID:** node-21<br>**Similarity:** 0.2795438054011855<br>**Text:** How is awareness being raised for Keytruda as a treatment option? Awareness is being raised through patient advocacy groups, clinical trial publications, and healthcare provider education programs.<br>

**Node ID:** node-20<br>**Similarity:** 0.27798582154825885<br>**Text:** Are there patient support programs available for Keytruda users? Yes, there are several patient support programs that provide financial assistance and counseling services.<br>

**Node ID:** node-22<br>**Similarity:** 0.2750527397610798<br>**Text:** Is there a community support group for patients undergoing treatment with Keytruda? Yes, several online and local support groups are available for sharing experiences and advice.<br>

**Node ID:** node-3<br>**Similarity:** 0.2684294247400273<br>**Text:** What should patients report immediately while on Keytruda treatment? Patients should report any new or worsening symptoms such as cough, chest pain, or changes in vision immediately.<br>

# COn reranker
ref  https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/rankLLM/

In [4]:
index = base_index

In [5]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankllm_rerank import RankLLMRerank

import pandas as pd
import torch
from IPython.display import display, HTML

In [6]:
# with_reranker = True

# query_str = "Can you tell me about the key concepts for keykundra"
# query_bundle = QueryBundle(query_str)
# # configure retriever
# retriever = VectorIndexRetriever(
#     index=index,
#     similarity_top_k=5,
# )
# retrieved_nodes = retriever.retrieve(query_bundle)
# retrieved_nodes.reverse()

# if with_reranker:
#         # configure reranker
#         reranker = RankLLMRerank(
#             model=model, top_n=reranker_top_n, window_size=window_size
#         )
#         retrieved_nodes = reranker.postprocess_nodes(
#             retrieved_nodes, query_bundle
#         )

#         # clear cache, rank_zephyr uses 16GB of GPU VRAM
#         del reranker
#         torch.cuda.empty_cache()

In [7]:
# %pip show rank-llm 

In [8]:
# %pip install llama-index-core
# %pip install llama-index-llms-openai
# %pip install llama-index-postprocessor-rankllm-rerank
# %pip install rank-llm


In [9]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankllm_rerank import RankLLMRerank

import pandas as pd
import torch
from IPython.display import display, HTML


def get_retrieved_nodes(
    query_str,
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=False,
    model="rank_zephyr",
    window_size=None,
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)
    retrieved_nodes.reverse()

    if with_reranker:
        # configure reranker
        reranker = RankLLMRerank(
            model=model, top_n=reranker_top_n#, window_size=window_size
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

        # clear cache, rank_zephyr uses 16GB of GPU VRAM
        del reranker
        torch.cuda.empty_cache()

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [10]:
new_nodes = get_retrieved_nodes(
    "Can you tell me about the key concepts for keykundra",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
    model="gpt",
)

ModuleNotFoundError: No module named 'rank_llm.result'

In [11]:
from rank_llm import *

In [17]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles?",
    vector_top_k=50,
    reranker_top_n=3,
    with_reranker=True,
    model="rank_zephyr",
    window_size=15,
)

visualize_retrieved_nodes(new_nodes)

TypeError: __init__() got an unexpected keyword argument 'window_size'